In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import requests

In [2]:
TRAIN = 'open'
TARGET = 'close'

In [3]:
url = "https://api.finmindtrade.com/api/v4/login"
account = {
    "user_id": "F74086250",
    "password": "Aeiou95048",
}
token = requests.post(url, data=account).json()['token']

url = "https://api.finmindtrade.com/api/v4/data"
parameter = {
    "dataset": "TaiwanStockPrice",
    "data_id": "2330",
    "start_date": "2020-01-01",
    "end_date": "2021-05-31",
    "token":token
}
data = pd.DataFrame(requests.get(url, params=parameter).json()["data"])
data[TRAIN] = data["Trading_money"]/data["Trading_Volume"]
data

,date,stock_id,Trading_Volume,Trading_money,open,max,min,close,spread,Trading_turnover
0,2020-01-02,2330,33282120,11224165450,337.243104,339.0,332.5,339.0,8.0,17160
1,2020-01-03,2330,42023268,14295651580,340.184195,345.0,335.5,339.5,0.5,20545
2,2020-01-06,2330,45677057,15210531318,333.001562,334.5,332.0,332.0,-7.5,22944
3,2020-01-07,2330,51746181,17053282011,329.556340,333.0,326.5,329.5,-2.5,22362
4,2020-01-08,2330,37913748,12484410088,329.284514,333.0,325.0,329.5,0.0,18228
...,...,...,...,...,...,...,...,...,...,...
335,2021-05-25,2330,35445350,20525932956,579.086762,584.0,573.0,583.0,15.0,39032
336,2021-05-26,2330,19555305,11433686898,584.684662,588.0,581.0,585.0,2.0,21034
337,2021-05-27,2330,70061002,40643804852,580.120234,582.0,573.0,582.0,-3.0,37545
338,2021-05-28,2330,30720737,18082265480,588.601292,592.0,582.0,590.0,8.0,31581


In [4]:
#切分Test集
test = data[data.date>'2021-02-01']
train = data[data.date<='2021-02-01']
#只要open high
train_set_x = train[TRAIN]
train_set_y = train[TARGET]

In [5]:
from sklearn.preprocessing import MinMaxScaler 
sc = MinMaxScaler(feature_range = (0, 1))
#需將資料做reshape的動作，使其shape為(資料長度,1) 
train_set_x= train_set_x.values.reshape(-1,1)
training_set_scaled_x = sc.fit_transform(train_set_x)
train_set_y= train_set_y.values.reshape(-1,1)
training_set_scaled_y = sc.fit_transform(train_set_y)

In [6]:
X_train = [] 
y_train = []
for i in range(10,len(train_set)):
    X_train.append(training_set_scaled_x[i-10:i-1, 0:1]) 
    y_train.append(training_set_scaled_y[i, 0]) 
X_train, y_train = np.array(X_train), np.array(y_train) 
X_train = np.reshape(X_train,(X_train.shape[0], X_train.shape[1], 1))

NameError: name 'train_set' is not defined

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout,BatchNormalization

In [ ]:
keras.backend.clear_session()
regressor = Sequential()
regressor.add(LSTM(units = 100, input_shape = (X_train.shape[1], 1)))
regressor.add(Dense(units = 1))
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
regressor.summary()

In [ ]:
history = regressor.fit(X_train, y_train, epochs = 150, batch_size = 16)
plt.title('train_loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.plot( history.history["loss"])

In [ ]:
dataset_total = pd.concat((train[TRAIN], test[TARGET]), axis = 0)
inputs = dataset_total[len(dataset_total) - len(test) - 10:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(10, len(inputs)):
    X_test.append(inputs[i-10:i-1, 0])
X_test = np.array(X_test)
#print(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
#print(X_test)
predicted_stock_price = regressor.predict(X_test)
print(predicted_stock_price)
#使用sc的 inverse_transform將股價轉為歸一化前
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
plt.plot(test[TARGET].values, color = 'black', label = 'Real TSMC Stock Price')
plt.plot(predicted_stock_price, color = 'green', label = 'Predicted TSMC Stock Price')
plt.title('TATA Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()
#plt.savefig('lstm_2330.png')